<a href="https://colab.research.google.com/github/showmik121/Genetivie_AI/blob/main/Genetive_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**sentimate analysis**





In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification")
pipe(["This restaurant is awesome", "This restaurant is awful"])

In [ ]:

from transformers import pipeline
pipe = pipeline("text-classification")
pipe(["This restaurant is awesome", "This restaurant is awful"])

In [ ]:
!pip install optimum[onnxruntime]

In [ ]:
sentences = ["This restaurant is awesome", "This restaurant is awful"]

from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForSequenceClassification

model_id = "SamLowe/roberta-base-go_emotions-onnx"
file_name = "onnx/model_quantized.onnx"

model = ORTModelForSequenceClassification.from_pretrained(model_id, file_name=file_name)
tokenizer = AutoTokenizer.from_pretrained(model_id)

onnx_classifier = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    top_k=None,
    function_to_apply="sigmoid",  # optional as is the default for the task
)

model_outputs = onnx_classifier(sentences)

print(model_outputs)

# test generation

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')
generator("Dhaka is a capital of Bangladesh.", do_sample=True, min_length=50)

In [ ]:
from transformers import pipeline,set_seed
generator = pipeline('text-generation', model='distilgpt2')
set_seed(42)
generator("Dhaka is capital of Bangladesh.", truncation=True, num_return_sequences=2)


# question answering

In [ ]:
from transformers import pipeline
nlp = pipeline("question-answering")
question="Where do I live?"
context="My name is Merve and I live in İstanbul."
nlp(question = question, context = context)


# Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,DistilBertTokenizer,DistilBertForSequenceClassification


In [ ]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

text="Hello, how are you?"
tokens=tokenizer.tokenize(text)
token_ids=tokenizer.convert_tokens_to_ids(tokens)
print("Tokens: ",tokens)
print("Token IDs: ",token_ids)

Tokens:  ['hello', ',', 'how', 'are', 'you', '?']
Token IDs:  [7592, 1010, 2129, 2024, 2017, 1029]


In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
text="Hello, how are you?"
tokens=tokenizer.tokenize(text)
token_ids=tokenizer.convert_tokens_to_ids(tokens)
print("Tokens: ",tokens)
print("Token IDs: ",token_ids)

Tokens:  ['Hello', ',', 'Ġhow', 'Ġare', 'Ġyou', '?']
Token IDs:  [31414, 6, 141, 32, 47, 116]


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text="Hello, how are you?"
tokens=tokenizer.tokenize(text)
token_ids=tokenizer.convert_tokens_to_ids(tokens)
print("Tokens: ",tokens)
print("Token IDs: ",token_ids)
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encoded_inputs=tokenizer(["hello, how are you?,I am fine,thank you."],
                         padding=True,
                         truncation=True,
                         return_tensors='pt')
print("Encoder inputs: ", encoded_inputs)

Tokens:  ['Hello', ',', 'Ġhow', 'Ġare', 'Ġyou', '?']
Token IDs:  [15496, 11, 703, 389, 345, 30]
Encoder inputs:  {'input_ids': tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029, 1010, 1045, 2572, 2986, 1010,
         4067, 2017, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
from pickle import decode_long
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
tex1=("Hello!,how are you?")
tex2=("I am fine,thank you.")
encoded_inputs=tokenizer.encode_plus(tex1,tex2,
                         padding='max_length',
                          add_special_tokens=True,
                          max_length=12,
                         truncation=True,
                         return_tensors='pt')
print("Encoder input IDs: ", encoded_inputs['input_ids'])
print("Token Type IDs: ",encoded_inputs['token_type_ids'])
print("Encoder attention masks: ", encoded_inputs['attention_mask'])
# decoded the inputs ids back
decode=tokenizer.decode(encoded_inputs['input_ids'][0],skip_special_tokens=False)
print(decode)

# Fine Tuning

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("imdb")
ds['train'][0]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

In [ ]:
tokenized_ds

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="./results", evaluation_strategy="epoch",learning_rate=2e-5,
                                  per_device_train_batch_size=16, # Changed 'pre_device_train_batch_size' to 'per_device_train_batch_size'
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=1,
                                  weight_decay=0.01)
training_args

In [ ]:
from transformers import AutoModelForSequenceClassification,Trainer
# load the pre-trained model
model=AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=2)

In [ ]:
# Initialize the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],

)

In [ ]:
# train the model
trainer.train()

In [ ]:
# evelute the model
trainer.evaluate()

In [ ]:
# save the mode
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-tokenizer")

In [ ]:
from transformers import pipeline

# Load fine-tuned model
classifier = pipeline("text-classification", model="fine_tuned_model", tokenizer="fine_tuned_tokenizer")

# Test on a new review
review = "The movie was absolutely fantastic! The characters were well-developed."
result = classifier(review)

print(result)  # Example: [{'label': 'LABEL_1', 'score': 0.9991}] (LABEL_1 = positive, LABEL_0 = negative)


# Image Generation

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
pip install diffusers transformers accelerate scipy

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
# model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"
# pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
# pipe = pipe.to(device)



1.   StableDiffusionPipeline: Loads the pre-trained Stable Diffusion model


2.   torch: Used for handling tensors and GPU acceleration


3.   matplotlib.pyplot: Used for displaying generated images

4.   torch.float16:Reduces VRAM usage (important for GPUs with less than 16GB of VRAM).





In [ ]:
pip show torch

In [ ]:
model_id1="dreamlike-art/dreamlike-diffusion-1.0"
model_id2="stabilityai/stable-diffusion-xl-base-1.0"
pipe1 = StableDiffusionPipeline.from_pretrained(model_id1, torch_dtype=torch.float16,use_safetensors=True)
pipe1 = pipe1.to(device)

In [ ]:

prompt = "dreamlikeart, a grungy woman with rainbow hair, travelling between dimensions, dynamic pose, happy, soft eyes and narrow chin, extreme bokeh, torn kawaii shirt and baggy jeans, In style of by Jordan Grimmer, crisp lines and color, complex background, particles, lines, wind, concept art, sharp focus, vivid colors"
image = pipe(prompt).images[0]

image.save("./result.jpg")


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
image

In [ ]:
print("[Prompt]: ",prompt)
plt.imshow(image);
plt.axis('off');

# Working with stable diffusion parameters

###negative prompting
###num_inference_steps
###height
###weight
###num_image_per_prompt

In [ ]:
def generate_image(pipe,prompt,params)
  image=pipe(prompt,**params).images
  num_images=len(image)
  if num_images>1
     fig,ax=plt.subplots(nrows=1,ncols=num_images)
     for i in range(num_images):
       ax[i].imshow(image[i])
       ax[i].axis('off')
  else:
    fig=plt.figure()
    plt.imshow(image[0]);
    plt.axis('off');
  plt.tight_layout()
  plt.show()



In [ ]:
prompt="dreamlikeart, a grungy woman with rainbow hair, travelling between dimensions, dynamic pose, happy, soft eyes and narrow chin, extreme bokeh, torn kawaii shirt and baggy jeans, In style of by Jordan Grimmer, crisp lines and color, complex background, particles, lines, wind, concept art, sharp focus, vivid colors"
params={"num_inference_steps":50,"height":512,"width":512}
generate_image(pipe,prompt,params)